In [ ]:
import os
import uuid
import random
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from google.colab import files
import shutil

In [ ]:
# Ensure a clean data directory
DATA_DIR = 'data'
if os.path.exists(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(DATA_DIR)

def generate_dimensions():
    # 1. System Dimension
    categories = ['Database', 'Web Service', 'Network', 'Application', 'Cloud']
    systems = pd.DataFrame([{
        'system_key': i,
        'system_name': f'System_{i}',
        'system_type': random.choice(categories),
        'description': f'Core {random.choice(categories)} infrastructure'
    } for i in range(1, 51)])

    # 2. Team Dimension
    regions = ['EMEA', 'APAC', 'AMER']
    teams = pd.DataFrame([{
        'team_key': i,
        'team_name': f'Team_{i}',
        'manager_name': f'Manager_{i}',
        'region': random.choice(regions)
    } for i in range(1, 11)])

    # 3. Severity Dimension
    severities = pd.DataFrame([
        {'severity_key': 1, 'severity_label': 'SEV1', 'description': 'Critical impact'},
        {'severity_key': 2, 'severity_label': 'SEV2', 'description': 'Major impact'},
        {'severity_key': 3, 'severity_label': 'SEV3', 'description': 'Minor impact'}
    ])

    return systems, teams, severities

systems_df, teams_df, severities_df = generate_dimensions()
print("Dimensions created successfully.")

Dimensions created successfully.


In [ ]:
def generate_time_and_assets(systems, teams):
    # 1. Time Dimension (2024-2025)
    dates = pd.date_range(start='2024-01-01', end='2025-12-31', freq='D')
    time_dim = pd.DataFrame([{
        'time_key': i + 1,
        'date_id': dt.date(),
        'year': dt.year,
        'quarter': (dt.month - 1) // 3 + 1,
        'month': dt.month,
        'month_name': dt.strftime('%B'),
        'day': dt.day,
        'day_of_week': dt.strftime('%A')
    } for i, dt in enumerate(dates)])

    # 2. Asset Dimension
    assets = pd.DataFrame([{
        'asset_key': i,
        'asset_id': f'ASSET_{uuid.uuid4().hex[:8].upper()}',
        'system_key': systems.sample(1).iloc[0]['system_key'],
        'team_key': teams.sample(1).iloc[0]['team_key'],
        'purchase_date': (datetime.now() - timedelta(days=random.randint(0, 2000))).date(),
        'lifecycle': random.choice(['New', 'Midlife', 'End-of-life']),
        'utilisation': round(random.uniform(30.0, 95.0), 2)
    } for i in range(1, 5001)])

    return time_dim, assets

time_df, assets_df = generate_time_and_assets(systems_df, teams_df)
print("Time and Asset data created.")

Time and Asset data created.


In [ ]:
def generate_all_facts(systems, teams, assets, time_dim):
    start_dt = datetime(2024, 1, 1)
    date_map = time_df.set_index('date_id')['time_key'].to_dict()

    # 1. Fact_Incidents (Break-fix Workflows)
    incident_rows = []
    severity_dist = [1]*5 + [2]*20 + [3]*75
    sla_targets = {1: 4, 2: 12, 3: 48}

    for _ in range(15000):
        sev = random.choice(severity_dist)
        creation_dt = start_dt + timedelta(days=random.randint(0, 700), hours=random.randint(0, 23))
        res_hours = np.random.exponential(scale=sla_targets[sev] * 0.7)
        incident_rows.append({
            'incident_id': f'INC_{uuid.uuid4().hex[:10].upper()}',
            'system_key': systems.sample(1).iloc[0]['system_key'],
            'team_key': teams.sample(1).iloc[0]['team_key'],
            'asset_key': assets.sample(1).iloc[0]['asset_key'],
            'severity_key': sev,
            'creation_time': creation_dt,
            'resolution_time': creation_dt + timedelta(hours=res_hours),
            'resolution_time_hours': round(res_hours, 2),
            'sla_breach_flag': 1 if res_hours > sla_targets[sev] else 0,
            'recurrence_flag': 1 if random.random() < 0.15 else 0,
            'time_key': date_map.get(creation_dt.date())
        })

    # 2. Fact_Changes (Service Transition)
    change_rows = []
    for _ in range(3000):
        impl_dt = start_dt + timedelta(days=random.randint(0, 700))
        change_rows.append({
            'change_id': f'CHG_{uuid.uuid4().hex[:10].upper()}',
            'system_key': systems.sample(1).iloc[0]['system_key'],
            'team_key': teams.sample(1).iloc[0]['team_key'],
            'implementation_time': impl_dt,
            'change_type': random.choice(['Standard', 'Normal', 'Emergency']),
            'success_flag': 1 if random.random() > 0.1 else 0,
            'linked_incident_count': random.randint(0, 3) if random.random() < 0.2 else 0,
            'time_key': date_map.get(impl_dt.date())
        })

    # 3. Fact_SLA (Availability Targets)
    sla_rows = []
    # Generate one SLA record per Month/System/Team combination
    for sys_key in systems['system_key'].unique():
        for team_key in teams['team_key'].unique():
            # Monthly snapshots for 2024
            for month_start in pd.date_range(start='2024-01-01', end='2025-12-01', freq='MS'):
                target = 99.9
                actual = max(95.0, 99.9 - np.random.exponential(0.5))
                sla_rows.append({
                    'time_key': date_map.get(month_start.date()),
                    'system_key': sys_key,
                    'team_key': team_key,
                    'target_resolution_hours': 24.0,
                    'actual_resolution_hours': round(random.uniform(18, 26), 2),
                    'sla_breach_percentage': round(100 - actual, 2)
                })

    return pd.DataFrame(incident_rows), pd.DataFrame(change_rows), pd.DataFrame(sla_rows)

inc_df, chg_df, sla_df = generate_all_facts(systems_df, teams_df, assets_df, time_df)
print(f"Generated Incidents ({len(inc_df)}), Changes ({len(chg_df)}), and SLA ({len(sla_df)}) records.")

Generated Incidents (15000), Changes (3000), and SLA (12000) records.


In [ ]:
# Save all to CSV
systems_df.to_csv(f'{DATA_DIR}/Dim_System.csv', index=False)
teams_df.to_csv(f'{DATA_DIR}/Dim_Team.csv', index=False)
severities_df.to_csv(f'{DATA_DIR}/Dim_Severity.csv', index=False)
time_df.to_csv(f'{DATA_DIR}/Dim_Time.csv', index=False)
assets_df.to_csv(f'{DATA_DIR}/Dim_Asset.csv', index=False)
inc_df.to_csv(f'{DATA_DIR}/Fact_Incidents.csv', index=False)
chg_df.to_csv(f'{DATA_DIR}/Fact_Changes.csv', index=False)
sla_df.to_csv(f'{DATA_DIR}/Fact_SLA.csv', index=False)

# Zip and Download
shutil.make_archive('Microsoft_COI_Project_Data', 'zip', DATA_DIR)
files.download('Microsoft_COI_Project_Data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>